In [2]:
import pandas as pd
import numpy as np
import os, sys, json, pickle, time
from collections import Counter






In [5]:
data.shape

(6309, 514)

In [3]:
with open('data/256_train.pickle','rb') as f:
  train_data = pickle.load(f)
  
#with open('../data/test.pickle','rb') as f:
#  test_data = pickle.load(f)
  
with open('data/256_validation.pickle','rb') as f:
  validation_data = pickle.load(f)

In [4]:
Xtrain, ytrain = train_data[:,1:-1], train_data[:,-1]
Xvalidation, yvalidation = validation_data[:,1:-1], validation_data[:,-1]

In [8]:
Xtrain.mean(axis = 0)

array([1.53066204e+00, 5.14027816e-02, 1.32250567e-02, 4.56507070e-01,
       7.26045891e-01, 1.28280417e-01, 2.56119716e-02, 2.09911528e-01,
       1.03420036e+00, 9.56364028e-01, 1.20645356e-01, 6.35285447e-03,
       2.37642915e-01, 1.85482771e-01, 6.04625347e-01, 1.67816492e+00,
       9.95088615e-01, 7.17660703e-01, 1.08147990e-01, 8.27619278e-01,
       2.15604233e+00, 2.33462632e-01, 5.04887310e-01, 3.39756328e-01,
       9.49469440e-03, 4.52922312e-03, 1.74971964e+00, 1.47275419e-01,
       1.61020933e-03, 7.52074362e-01, 1.07641060e-02, 5.15234245e-01,
       1.09267112e+00, 1.43472755e+00, 9.69655572e-01, 2.51229263e-02,
       1.93526256e+00, 6.56361926e-02, 1.00172718e-01, 2.06189152e-01,
       3.60483478e-01, 9.00159552e-02, 8.54724612e-02, 2.11587925e-01,
       9.78757720e-02, 1.65500002e+00, 1.25067087e+00, 1.46429121e+00,
       9.58540629e-02, 1.36638811e-01, 4.62270243e-01, 3.32366971e-01,
       6.37116509e-01, 2.36540875e-01, 1.33387652e+00, 1.22529950e+00,
      

# Forward feature selection
---

Let first try what they did in the exercises. So forward feature selection:
Lets (as in the exercise) try to find 5 features that are "good" using a random sample of 10000.

---

In [6]:
#numpy.random.seed(1)
rnd_list = np.random.randint(0, train_data.shape[0],10000)
rnd_trn, rnd_val = rnd_list[:8000], rnd_list[8000:]

Xtrn, ytrn = Xtrain[rnd_trn], ytrain[rnd_trn]
Xval, yval = Xtrain[rnd_val], ytrain[rnd_val]

In [8]:
start = time.time()

good_features = []
val_scores = []

feature_list = list(range(Xtrain.shape[1]))

for j in range(1,6):
    scores = []
    for i in feature_list:
        feature_slice = good_features+[i]
        X = Xtrn[:,feature_slice]
        model = KNeighborsClassifier(n_neighbors=10)
        model.fit(X, ytrn)
        scores.append(1- model.score(Xval[:,feature_slice],yval))
    
    best_ind = np.argmin(scores)
    best_feature = feature_list[best_ind]
    
    val_scores.append(scores[best_ind])
    good_features.append(best_feature)
    del feature_list[best_ind]
    print(good_features)
    
print('Seconds to run: {}'.format(time.time() - start))

[146]
[146, 102]
[146, 102, 110]
[146, 102, 110, 165]
[146, 102, 110, 165, 177]
Seconds to run: 79.99695777893066


In [15]:
val_scores

[0.9755, 0.968, 0.967, 0.9635, 0.963]

### Conclusion

It seems that 5 features is not enough to get a proper score. maybe 10000 is also not enough to do this. 
Also the exercise data had 54 features, we have 256, so maybe crank up the number of features desired. 

---
# PCA

Also to furter reduce the number of features, lets look at PCA

----

In [9]:
from sklearn.decomposition import PCA

pca= PCA()
pca.fit(Xtrain)

PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [23]:
print(pca.explained_variance_ratio_.cumsum()[65])
np.where(pca.explained_variance_ratio_.cumsum()> 0.90)

0.8450310831930141


(array([106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
        119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131,
        132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144,
        145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157,
        158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170,
        171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183,
        184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196,
        197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209,
        210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222,
        223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235,
        236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248,
        249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261,
        262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274,
        275, 276, 277, 278, 279, 280, 281, 282, 283

About 85% of the variance in data can be expressed using only 44 principle components. This seems like a nice number.

So lets transform the data and run a couple classifiers.

In [11]:
TXtrain = pca.transform(Xtrain)[:,:65]
TXvalidation = pca.transform(Xvalidation)[:,:65]

In [9]:
# K nearest neighbor classifier
# Score is the mean accuracy

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 15)
knn.fit(Xtrain,ytrain)
knn.score(Xvalidation, yvalidation)

0.23189094943731178

In [10]:
# Random Forest Classifier
# Score is the mean accuracy

from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators = 10)
rfc.fit(Xtrain, ytrain)
rfc.score(Xvalidation,yvalidation)

0.15596766524013314

In [11]:
# A multilayer Preceptron
# Score is the mean accuracy again

from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier((200,150))
mlp.fit(Xtrain, ytrain)
mlp.score(Xvalidation,yvalidation)

0.2664447614518941

In [12]:
# Naive Bayes Classifier
# Score is the mean accuracy

from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(Xtrain,ytrain)
gnb.score(Xvalidation,yvalidation)

0.11760976382944999